# **Configuração Inicial**

## Importação das bibliotecas

In [1]:
# Importação das bibliotecas e funções a serem utilizadas no códigos.
from datetime import datetime
import db_connector as db

# **Testes de Conexão**

## Testando Operação de Leitura

In [2]:
s = db.spark_conn()
df = s.read()
df.show()
s.stop_session()

+--------------------+--------+------------------+-----------+-------------+-------------------+--------------+---------+------------+--------------------+-------------------+--------------+-------------------+--------------------+--------------------------+-------------------------------+----------------------+-----------------+----------------------------+---------------------------------+------------------------+----------------+-------------------------+----------------------------+---------------+---------+-------------------+--------------+
|                 _id|baseFare|destinationAirport|elapsedDays|fareBasisCode|         flightDate|isBasicEconomy|isNonStop|isRefundable|               legId|         searchDate|seatsRemaining|segmentsAirlineCode| segmentsAirlineName|segmentsArrivalAirportCode|segmentsArrivalTimeEpochSeconds|segmentsArrivalTimeRaw|segmentsCabinCode|segmentsDepartureAirportCode|segmentsDepartureTimeEpochSeconds|segmentsDepartureTimeRaw|segmentsDistance|segmentsDur

## Testando Operação de Escrita

In [71]:
s = db.spark_conn()
df = s.read()

# Definindo o schema do novo registro.
schema = df.schema

# Definindo os dados para o novo registro.
data = [
    {
        "_id": "a1",
        "baseFare": 260.47,
        "destinationAirport": "BOS",
        "elapsedDays": 0,
        "fareBasisCode": "L0AJZNN1",
        "flightDate": datetime.fromisoformat("2022-04-17T00:00:00"),
        "isBasicEconomy": False,
        "isNonStop": False,
        "isRefundable": False,
        "legId": "721d9a2f66fe479e7c17b13e7ae0bb15",
        "searchDate": datetime.fromisoformat("2022-04-16T00:00:00"),
        "seatsRemaining": 1,
        "segmentsAirlineCode": "AA||AA",
        "segmentsAirlineName": "American Airlines||American Airlines",
        "segmentsArrivalAirportCode": "CLT||BOS",
        "segmentsArrivalTimeEpochSeconds": "1650212880||1650224280",
        "segmentsArrivalTimeRaw": "2022-04-17 12:28:00||2022-04-17 15:38:00",
        "segmentsCabinCode": None,
        "segmentsDepartureAirportCode": "ATL||CLT",
        "segmentsDepartureTimeEpochSeconds": "1650207720||1650216420",
        "segmentsDepartureTimeRaw": "2022-04-17 11:02:00||2022-04-17 13:27:00",
        "segmentsDistance": None,
        "segmentsDurationInSeconds": None,
        "segmentsEquipmentDescription": "Canadair Regional Jet 900||Airbus A321",
        "startingAirport": "ATL",
        "totalFare": 302.11,
        "totalTravelDistance": 956,
        "travelDuration": "PT4H36M"
    }
]

new_df = s.createDataFrame(data, schema=schema)

s.write(new_df)

s.stop_session()

Para remoção do registro adicionado anteriormente, porém, o uso deve ser no terminal mongosh do próprio mongoDB.

- `%use ProjetoPMD`
- `%db.itineraries.deleteOne({ _id: "a1" })`

# **Consulta 01**

## Enunciado

Partindo de um determinado aeroporto, com base nas informações coletadas nos 3 meses anteriores e num raio Y de milhas do aeroporto, listar a quantidade de voos para cada um dos aeroportos atendidos por ele, o preço médio da passagem e a duração média desses voos, além de mostrar quais foram as empresas aérea em operação.

## Execução

In [4]:
s = db.spark_conn()

df = s.read(columns=[
    "flightDate",
    "totalTravelDistance",
    "travelDuration",
    "totalFare",
    "startingAirport",
    "destinationAirport",
    "segmentsAirlineName"
    ])

df.show()

s.stop_session()

+-------------------+-------------------+--------------+---------+---------------+------------------+--------------------+
|         flightDate|totalTravelDistance|travelDuration|totalFare|startingAirport|destinationAirport| segmentsAirlineName|
+-------------------+-------------------+--------------+---------+---------------+------------------+--------------------+
|2022-04-16 21:00:00|                947|       PT2H29M|    248.6|            ATL|               BOS|               Delta|
|2022-04-16 21:00:00|                947|       PT2H30M|    248.6|            ATL|               BOS|               Delta|
|2022-04-16 21:00:00|                947|       PT2H30M|    248.6|            ATL|               BOS|               Delta|
|2022-04-16 21:00:00|                947|       PT2H32M|    248.6|            ATL|               BOS|               Delta|
|2022-04-16 21:00:00|                947|       PT2H34M|    248.6|            ATL|               BOS|               Delta|
|2022-04-16 21:0

# **Consulta 02**

## Enunciado

Listar o preço médio por distância viajada de cada empresa aérea, agrupando por intervalos de distância a cada 500 milhas e separando as viagens entre econômica e não econômica.

## Execução

In [ ]:
s = db.spark_conn()

s.read(columns=[

    ])

s.stop_session()